In [1]:
# cell 1

import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# 1. Load count matrix using TPM
data_path = '/Users/esrataner/Documents/DATA1030/count_matrix_120'
file_list = [f for f in os.listdir(data_path) if f.endswith('.tsv')]

def read_tpm_file(filename):
    df = pd.read_csv(os.path.join(data_path, filename), sep='\t',
                     usecols=['gene_id', 'TPM'])
    df = df.set_index('gene_id')
    sample_id = filename.replace('.tsv', '')
    df.columns = [sample_id]
    return df

dfs = [read_tpm_file(f) for f in file_list]
merged = pd.concat(dfs, axis=1)  # genes × samples
print("Raw merged shape:", merged.shape)

# 2. remove all spike-in genes 
# these are technical control genes - not needed for ML 
spikein_mask = merged.index.str.lower().str.contains(
    "spikein|ercc"
)

print("Spike-in genes detected:", spikein_mask.sum())

merged = merged.loc[~spikein_mask]
print("After removing spike-ins:", merged.shape)

# 3. Remove all-zero genes
zero_across_all = (merged == 0).all(axis=1)
print("Genes zero everywhere:", zero_across_all.sum())

expr = merged[~zero_across_all]
print("After removing all-zero genes:", expr.shape)

# 4. log1p(TPM) transform — biological normalization
expr_log = np.log1p(expr)

# NOTE: try graphing log1p and log 
# depends on if data is close to zero or not 

Raw merged shape: (59526, 120)
Spike-in genes detected: 97
After removing spike-ins: (59429, 120)
Genes zero everywhere: 14289
After removing all-zero genes: (45140, 120)


In [2]:
# cell 2

meta = pd.read_csv(
    '/Users/esrataner/Documents/DATA1030/tsv/experiment_report_2025_120.tsv',
    sep='\t', skiprows=1
)

# extract sex
meta['Sex'] = meta['Biosample summary'].str.extract(r'(?i)\b(female|male)\b')
meta['Sex'] = meta['Sex'].str.lower()

# parse numeric age
def parse_age(a):
    if pd.isna(a):
        return None
    nums = re.findall(r'\d+', str(a))
    return int(nums[0]) if nums else None

meta['Age'] = meta['Biosample age'].apply(parse_age)

# Age bins
def age_bin(x):
    if pd.isna(x): return None
    if x < 65: return "60-64"
    if x < 70: return "65-69"
    if x < 75: return "70-74"
    if x < 80: return "75-79"
    if x < 85: return "80-84"
    if x < 90: return "85-89"
    return "90+"

meta['Age_Ordinal'] = meta['Age'].apply(age_bin)

age_order = ["60-64","65-69","70-74","75-79","80-84","85-89","90+"]
meta['Age_Ordinal'] = pd.Categorical(
    meta['Age_Ordinal'], categories=age_order, ordered=True
)

# FIX!!  Extract *all* ENCFF file IDs (not just first)
# Find ALL ENCFF IDs
meta['File_IDs'] = meta['Files'].str.findall(r'ENCFF\w+')

# Expand metadata so each row contains ONE ENCFF ID
meta_expanded = meta.explode('File_IDs')

# Keep only relevant fields
meta_expanded = meta_expanded[
    ['Accession', 'Sex', 'Age_Ordinal', 'Biosample accession', 'File_IDs']
]

# Drop rows missing file IDs
meta_expanded = meta_expanded.dropna(subset=['File_IDs'])

print("Expanded metadata shape:", meta_expanded.shape)
# Expanded metadata shape: (1220, 5)

# exp 
# in the raw ENCODE metadata, each biological sample (ENCSR) has multiple ENCFF files 
# (technical sequencing files: BAMs, fastqs, quantifications, etc)

# After expanding, u get 1 row per ENCFF file
# so if each ENCSR has ~10 ENCFF files ->  120 biological samples become ~1220 rows.
# so 1220 is NOT 120 samples - it is technical files, NOT samples

# MOST IMPORTANTLY - do NOT need GroupShuffleSplit
# 1. each ENCSR is represented only once
# 2. no ENCFF technical replicates remain#
# 3. df_full has 1 row per biological sample





Expanded metadata shape: (1220, 5)


In [3]:
# cell 3 

# List of ENCFF IDs found in expression matrix
expr_samples = expr_log.columns.tolist()

# Match metadata rows where File_IDs map to expression samples
meta_matched = meta_expanded[meta_expanded['File_IDs'].isin(expr_samples)].copy()
print("Matched metadata rows:", meta_matched.shape)

# Step 1: Relabel expression matrix columns (ENCFF → ENCSR)
rename_dict = dict(zip(meta_matched['File_IDs'], meta_matched['Accession']))
expr_labeled = expr_log.rename(columns=rename_dict)

# Step 2: Identify replicate sample IDs (appear twice)
replicate_accessions = expr_labeled.columns[
    expr_labeled.columns.duplicated()
].unique()

print("Replicate sample accessions:", replicate_accessions.tolist())

# Step 3: DROP the replicate sample completely
expr_no_reps = expr_labeled.drop(columns=list(replicate_accessions))
meta_no_reps = meta_matched[
    ~meta_matched['Accession'].isin(replicate_accessions)
].copy()

print("Expression shape after removing replicates:", expr_no_reps.shape)
print("Metadata shape after removing replicates:", meta_no_reps.shape)

# Step 4: Align expression with metadata
expr_T = expr_no_reps.T
expr_T.index.name = 'Accession'
expr_T = expr_T.reset_index()

# Deduplicate final metadata
meta_unique = meta_no_reps.drop_duplicates(subset=['Accession'])

# Step 5: Merge into final ML-ready dataset
df_full = expr_T.merge(
    meta_unique.drop(columns=['File_IDs']),
    on='Accession',
    how='inner'
).set_index('Accession')

print("Final merged dataset shape:", df_full.shape)
print(df_full.head())


Matched metadata rows: (120, 5)
Replicate sample accessions: []
Expression shape after removing replicates: (45140, 120)
Metadata shape after removing replicates: (120, 5)
Final merged dataset shape: (120, 45143)
             13023     26893  30031  30958     30964  ENSG00000000003.14  \
Accession                                                                  
ENCSR800PJQ    0.0  0.506818    0.0    0.0  0.000000            1.348073   
ENCSR133PLR    0.0  0.000000    0.0    0.0  2.324347            1.340250   
ENCSR418WMG    0.0  0.000000    0.0    0.0  0.000000            1.181727   
ENCSR013HWB    0.0  0.438255    0.0    0.0  0.000000            1.313724   
ENCSR693KOP    0.0  0.000000    0.0    0.0  0.000000            1.081805   

             ENSG00000000005.5  ENSG00000000419.12  ENSG00000000457.13  \
Accession                                                                
ENCSR800PJQ           0.000000            2.157559            1.444563   
ENCSR133PLR           0.122218  

In [4]:
# cell 4 

# TRAIN / VAL / TEST SPLIT
from sklearn.model_selection import train_test_split

# (1) Define features + target
X = df_full.drop(columns=["Sex", "Biosample accession"]) #string carried over 
y = df_full["Sex"].map({"female": 0, "male": 1}).astype(int)

y = y.astype(int)

# (2) First split: Train vs Temp (Val+Test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# (3) Split Temp into Validation + Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

# Print shapes
print("Dataset Splits:")
print("Train:", X_train.shape, "Target:", y_train.shape)
print("Val:  ", X_val.shape,   "Target:", y_val.shape)
print("Test: ", X_test.shape,  "Target:", y_test.shape)


Dataset Splits:
Train: (84, 45141) Target: (84,)
Val:   (18, 45141) Target: (18,)
Test:  (18, 45141) Target: (18,)


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier
import numpy as np

# -------------------------------
# Compute scale_pos_weight
# -------------------------------
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / pos
print(f"scale_pos_weight = {scale_pos_weight:.4f}")

# -------------------------------
# Preprocessor
# -------------------------------
gene_cols = [c for c in X_train.columns if c != "Age_Ordinal"]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), gene_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), ["Age_Ordinal"])
])

preprocessor.fit(X_train)

X_train_trans = preprocessor.transform(X_train)
X_val_trans   = preprocessor.transform(X_val)

# -------------------------------
# Early-Stopping XGBoost
# -------------------------------
early_model = XGBClassifier(
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.2,
    eval_metric="logloss",
    tree_method="hist",
    early_stopping_rounds=10,
    random_state=42,
    scale_pos_weight=scale_pos_weight
)

early_model.fit(
    X_train_trans, y_train,
    eval_set=[(X_val_trans, y_val)],
    verbose=True
)

best_iter = early_model.best_iteration
print("Best n_estimators:", best_iter)


scale_pos_weight = 3.0000
[0]	validation_0-logloss:0.66791
[1]	validation_0-logloss:0.64257
[2]	validation_0-logloss:0.63504
[3]	validation_0-logloss:0.62314
[4]	validation_0-logloss:0.59624
[5]	validation_0-logloss:0.59290
[6]	validation_0-logloss:0.58634
[7]	validation_0-logloss:0.56521
[8]	validation_0-logloss:0.55086
[9]	validation_0-logloss:0.53728
[10]	validation_0-logloss:0.53500
[11]	validation_0-logloss:0.51820
[12]	validation_0-logloss:0.49902
[13]	validation_0-logloss:0.48449
[14]	validation_0-logloss:0.49170
[15]	validation_0-logloss:0.48403
[16]	validation_0-logloss:0.47638
[17]	validation_0-logloss:0.47090
[18]	validation_0-logloss:0.46432
[19]	validation_0-logloss:0.45436
[20]	validation_0-logloss:0.44672
[21]	validation_0-logloss:0.43318
[22]	validation_0-logloss:0.42612
[23]	validation_0-logloss:0.43208
[24]	validation_0-logloss:0.42850
[25]	validation_0-logloss:0.42556
[26]	validation_0-logloss:0.41823
[27]	validation_0-logloss:0.41624
[28]	validation_0-logloss:0.4222

In [6]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

def make_xgb_pipeline(n_estimators, random_state=42):
    return Pipeline(steps=[
        ("preprocessor", preprocessor),  
        ("model", XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            random_state=random_state,
            n_estimators=n_estimators,
            scale_pos_weight=scale_pos_weight
        ))
    ])


In [7]:
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import average_precision_score, classification_report

param_distributions = {
    "model__max_depth": [2, 3, 4, 5],
    "model__learning_rate": uniform(0.01, 0.1),
    "model__subsample": uniform(0.6, 0.4),
    "model__colsample_bytree": uniform(0.1, 0.5),
    "model__reg_alpha": uniform(0, 2),
    "model__reg_lambda": uniform(0, 2),
}

xgb_pipe = make_xgb_pipeline(best_iter)

search = RandomizedSearchCV(
    estimator=xgb_pipe,
    param_distributions=param_distributions,
    n_iter=40,
    scoring="average_precision",
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

search.fit(X_train, y_train)

print("\nBest CV PR-AUC:", search.best_score_)
print("\nBest Hyperparameters:")
for k, v in search.best_params_.items():
    print(f"  {k}: {v}")

best_rscv_params = search.best_params_


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END model__colsample_bytree=0.14998745790900145, model__learning_rate=0.05592488919658672, model__max_depth=2, model__reg_alpha=1.2022300234864176, model__reg_lambda=1.416145155592091, model__subsample=0.608233797718321; total time=   3.0s
[CV] END model__colsample_bytree=0.14998745790900145, model__learning_rate=0.05592488919658672, model__max_depth=2, model__reg_alpha=1.2022300234864176, model__reg_lambda=1.416145155592091, model__subsample=0.608233797718321; total time=   3.1s
[CV] END model__colsample_bytree=0.14998745790900145, model__learning_rate=0.05592488919658672, model__max_depth=2, model__reg_alpha=1.2022300234864176, model__reg_lambda=1.416145155592091, model__subsample=0.608233797718321; total time=   3.3s
[CV] END model__colsample_bytree=0.14998745790900145, model__learning_rate=0.05592488919658672, model__max_depth=2, model__reg_alpha=1.2022300234864176, model__reg_lambda=1.416145155592091, model__subsam

/Users/esrataner/miniforge3/envs/data1030/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END model__colsample_bytree=0.24607232426760908, model__learning_rate=0.046636184329369175, model__max_depth=3, model__reg_alpha=0.1812128690656416, model__reg_lambda=1.2367720186661746, model__subsample=0.7529847965068651; total time=   4.5s
[CV] END model__colsample_bytree=0.5744427686266667, model__learning_rate=0.10656320330745593, model__max_depth=3, model__reg_alpha=0.7708330050798322, model__reg_lambda=0.03193250444042839, model__subsample=0.6923575302488596; total time=   4.3s
[CV] END model__colsample_bytree=0.5744427686266667, model__learning_rate=0.10656320330745593, model__max_depth=3, model__reg_alpha=0.7708330050798322, model__reg_lambda=0.03193250444042839, model__subsample=0.6923575302488596; total time=   4.4s
[CV] END model__colsample_bytree=0.5744427686266667, model__learning_rate=0.10656320330745593, model__max_depth=3, model__reg_alpha=0.7708330050798322, model__reg_lambda=0.03193250444042839, model__subsample=0.6923575302488596; total time=   5.1s
[CV] END mo

In [8]:
# Combine Train + Val
X_train_full = pd.concat([X_train, X_val], axis=0)
y_train_full = pd.concat([y_train, y_val], axis=0)

# Rebuild model using best params
final_rscv_pipe = make_xgb_pipeline(best_iter)
final_rscv_pipe.set_params(**best_rscv_params)

final_rscv_pipe.fit(X_train_full, y_train_full)

# Test performance
y_test_proba = final_rscv_pipe.predict_proba(X_test)[:, 1]
y_test_pred  = final_rscv_pipe.predict(X_test)

test_ap = average_precision_score(y_test, y_test_proba)

print("\nFINAL TEST RESULTS (RSCV)")
print(f"TEST PR-AUC: {test_ap:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Save best params
pd.Series(best_rscv_params).to_json("rscv_best_params.json")



FINAL TEST RESULTS (RSCV)
TEST PR-AUC: 0.7708

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.75      0.75      0.75         4

    accuracy                           0.89        18
   macro avg       0.84      0.84      0.84        18
weighted avg       0.89      0.89      0.89        18



In [9]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pre = final_rscv_pipe.named_steps["preprocessor"]
model = final_rscv_pipe.named_steps["model"]

X_test_trans = pre.transform(X_test)
feature_names = pre.get_feature_names_out()

explainer = shap.TreeExplainer(model)
shap_vals = explainer.shap_values(X_test_trans)

if isinstance(shap_vals, list):
    shap_vals = shap_vals[1]

# ---------------------------
# Global SHAP ranking
# ---------------------------
mean_abs_shap = np.abs(shap_vals).mean(axis=0)

shap_ranked = (
    pd.DataFrame({"gene": feature_names, "mean_abs_shap": mean_abs_shap})
    .sort_values("mean_abs_shap", ascending=False)
)

shap_ranked.to_csv("rscv_shap_global_ranking.csv", index=False)

print("\nTop 10 features:")
display(shap_ranked.head(10))

# ---------------------------
# Directional SHAP
# ---------------------------
male_idx = np.where(y_test == 1)[0]
female_idx = np.where(y_test == 0)[0]

male_shap = shap_vals[male_idx].mean(axis=0)
female_shap = shap_vals[female_idx].mean(axis=0)

shap_diff = male_shap - female_shap

shap_direction = pd.DataFrame({
    "gene": feature_names,
    "male_shap": male_shap,
    "female_shap": female_shap,
    "difference_male_minus_female": shap_diff
})

shap_direction.sort_values("difference_male_minus_female", ascending=False).to_csv(
    "rscv_shap_directional.csv", index=False
)

print("\nTop Masculinizing Genes:")
display(shap_direction.sort_values("difference_male_minus_female", ascending=False).head(10))

print("\nTop Feminizing Genes:")
display(shap_direction.sort_values("difference_male_minus_female", ascending=True).head(10))



Top 10 features:


,gene,mean_abs_shap
4256,num__ENSG00000114374.12,0.163959
1018,num__ENSG00000067646.11,0.051228
22845,num__ENSG00000227494.2,0.049457
18783,num__ENSG00000207445.1,0.046965
29119,num__ENSG00000241859.6,0.046816
2100,num__ENSG00000099715.14,0.043909
26035,num__ENSG00000233864.7,0.042355
6127,num__ENSG00000129824.15,0.039892
9681,num__ENSG00000154620.5,0.028580
13895,num__ENSG00000176728.7,0.023044



Top Masculinizing Genes:


,gene,male_shap,female_shap,difference_male_minus_female
4256,num__ENSG00000114374.12,0.097151,-0.087838,0.184989
29119,num__ENSG00000241859.6,0.035427,-0.039419,0.074846
1018,num__ENSG00000067646.11,0.034306,-0.039714,0.074020
22845,num__ENSG00000227494.2,0.034868,-0.038178,0.073046
2100,num__ENSG00000099715.14,0.026184,-0.037279,0.063462
26035,num__ENSG00000233864.7,0.027930,-0.035197,0.063127
6127,num__ENSG00000129824.15,0.030543,-0.027004,0.057548
9681,num__ENSG00000154620.5,0.022138,-0.022133,0.044272
28139,num__ENSG00000238067.1,0.019781,-0.015817,0.035598
13895,num__ENSG00000176728.7,0.019057,-0.015413,0.034470



Top Feminizing Genes:


,gene,male_shap,female_shap,difference_male_minus_female
2968,num__ENSG00000104853.15,-0.016284,0.000693,-0.016977
2990,num__ENSG00000104904.12,-0.008036,-0.000087,-0.007949
21694,num__ENSG00000225231.1,-0.008116,-0.001390,-0.006726
17106,num__ENSG00000197980.12,-0.005903,-0.000091,-0.005812
31617,num__ENSG00000251079.6,-0.002074,0.003478,-0.005552
30556,num__ENSG00000248757.2,-0.005251,0.000285,-0.005536
4763,num__ENSG00000118246.13,-0.006236,-0.000708,-0.005528
2948,num__ENSG00000104760.16,-0.010169,-0.004797,-0.005372
13940,num__ENSG00000176945.16,-0.005333,-0.000153,-0.005180
17580,num__ENSG00000200320.1,-0.004914,-0.000226,-0.004689


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import shap

seed_results = []
shap_records = []
random_states = [0, 1, 2, 3, 4]

for seed in random_states:
    print(f"\n--- SEED {seed} ---")

    # 1. New split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.30, random_state=seed, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, random_state=seed, stratify=y_temp
    )

    preprocessor.fit(X_train)

    # 2. Rebuild final RSCV model for this seed
    model_seed = make_xgb_pipeline(best_iter, random_state=seed)
    model_seed.set_params(**best_rscv_params)

    X_train_full = pd.concat([X_train, X_val])
    y_train_full = pd.concat([y_train, y_val])

    model_seed.fit(X_train_full, y_train_full)

    # 3. Evaluate
    y_test_proba = model_seed.predict_proba(X_test)[:, 1]
    test_ap = average_precision_score(y_test, y_test_proba)

    # 4. SHAP values
    X_test_trans = preprocessor.transform(X_test)
    expl = shap.TreeExplainer(model_seed.named_steps["model"])
    sv = expl.shap_values(X_test_trans)
    if isinstance(sv, list):
        sv = sv[1]

    shap_df = pd.DataFrame(sv, columns=preprocessor.get_feature_names_out())
    shap_df["seed"] = seed
    shap_records.append(shap_df)

    seed_results.append({"seed": seed, "test_ap": test_ap})
    print(f"Test AP = {test_ap:.4f}")

# Save seed metrics
seed_df = pd.DataFrame(seed_results)
seed_df.to_csv("rscv_seed_metrics.csv", index=False)

# SHAP stability
shap_all = pd.concat(shap_records).drop(columns=["seed"])
shap_stability = shap_all.abs().mean().sort_values(ascending=False)
shap_stability.to_csv("rscv_shap_stability.csv")

print("\nSaved:")
print(" rscv_seed_metrics.csv")
print(" rscv_shap_stability.csv")



--- SEED 0 ---
Test AP = 0.5792

--- SEED 1 ---
Test AP = 0.6690

--- SEED 2 ---
Test AP = 0.9500

--- SEED 3 ---
Test AP = 0.7333

--- SEED 4 ---
Test AP = 0.9167

Saved:
 rscv_seed_metrics.csv
 rscv_shap_stability.csv
